In [156]:
import pandas as pd

## load in files

In [157]:
block = pd.read_csv("/Users/oliviafiol/Dropbox/CDS-2019-AlbanyHub/ToDatabase/full_blockgroup.csv")
address_counts = pd.read_csv("/Users/oliviafiol/Dropbox/CDS-2019-AlbanyHub/Processed-Data/address_count_per_blockgroup_gas.csv")
address_count_block = pd.read_csv("/Users/oliviafiol/Dropbox/CDS-2019-AlbanyHub/Processed-Data/address_count_per_blockgroup_housingproject_gas.csv")
avg_ccf = pd.read_csv("/Users/oliviafiol/Dropbox/CDS-2019-AlbanyHub/Processed-Data/avgccf_blockgroup_gas.csv")
avg_ccf_block = pd.read_csv("/Users/oliviafiol/Dropbox/CDS-2019-AlbanyHub/Processed-Data/avg_ccf_per_blockgroup.csv")

In [158]:
avg_ccf.head()

,Tract,BlockGroup,address_count,mean_ccf
0,1403,1,2,30.216667
1,600,2,2,28.090498
2,1100,2,8,15.199208
3,200,1,6,22.708163
4,10700,2,14,28.560092


In [159]:
avg_ccf_block.head()

,Tract,BlockGroup,avg_ccf,numrecords
0,10700.0,2.0,21.807457,28482
1,1403.0,1.0,23.884804,15287
2,10900.0,3.0,8.829175,521
3,100.0,1.0,20.899399,7654
4,10601.0,2.0,23.165816,8624


In [160]:
df = (block[["total_vacdata", "block_group", "tract"]]).rename(columns = {"tract":"Tract","block_group":"BlockGroup"})

In [161]:
df_full = df.merge(address_counts, on = ["Tract", "BlockGroup"], how = "inner")
df_full = df_full.merge(address_count_block, on = ['Tract', "BlockGroup"], how = "left")
df_full = df_full.merge(avg_ccf, on = ['Tract', "BlockGroup"], how = "left")
df_full = df_full.merge(avg_ccf_block, on = ['Tract', "BlockGroup"], how = "left")

In [162]:
df_full = df_full.fillna(0)

In [163]:
df_full = df_full.rename(columns = {"address_count_x":"utilities_address_count", "address_count_y":"projects_address_count"})

In [164]:
df_full["nonproj_address_count"] = df_full["utilities_address_count"] - df_full["projects_address_count"]

In [165]:
df_full["proj_pct"] = df_full["projects_address_count"] / df_full["utilities_address_count"] * 100

In [166]:
df_full["nonproj_pct"] = 100 - df_full["proj_pct"]

In [167]:
df_full = df_full.drop("address_count", axis = 1)

In [168]:
df_full = df_full.rename(columns = {"mean_ccf":"project_mean_ccf", "avg_ccf":"nonproj_mean_ccf"}).drop("numrecords", axis = 1)

In [169]:
df_full = df_full.rename(columns = {"total_vacdata":"total_properties"})

In [170]:
cols = list(df_full)
cols.insert(0, cols.pop(cols.index('Tract')))
cols.insert(1, cols.pop(cols.index("BlockGroup")))
cols.insert(5, cols.pop(cols.index("nonproj_address_count")))
cols.insert(6, cols.pop(cols.index("proj_pct")))
cols.insert(7, cols.pop(cols.index("nonproj_pct")))
df_full = df_full.loc[:, cols]

In [171]:
sum_props = df_full["total_properties"].sum()
sum_utilities = df_full["utilities_address_count"].sum()
sum_projects = df_full["projects_address_count"].sum()
sum_nonproj = df_full["nonproj_address_count"].sum()

In [172]:
df_full = df_full.append({"total_properties" : sum_props, "utilities_address_count" : sum_utilities,
                         "projects_address_count" : sum_projects, "nonproj_address_count" : sum_nonproj}, ignore_index =  True)

In [173]:
df_full.loc[63, "Tract"] = "Totals"
df_full.loc[63, "proj_pct"] = sum_projects/sum_utilities*100
df_full.loc[63, "nonproj_pct"] = sum_nonproj/sum_utilities*100
df_full.loc[63, "project_mean_ccf"] = 23.8300479488546
df_full.loc[63, 'nonproj_mean_ccf'] = 24.7935494099324


In [174]:
df_full.tail()

,Tract,BlockGroup,total_properties,utilities_address_count,projects_address_count,nonproj_address_count,proj_pct,nonproj_pct,project_mean_ccf,nonproj_mean_ccf
59,11400,2.0,181.0,98.0,2.0,96.0,2.040816,97.959184,18.807487,21.602568
60,11600,1.0,649.0,22.0,0.0,22.0,0.000000,100.000000,0.000000,20.139077
61,20300,2.0,616.0,1.0,0.0,1.0,0.000000,100.000000,0.000000,5.384615
62,20403,2.0,300.0,3.0,0.0,3.0,0.000000,100.000000,0.000000,5.935484
63,Totals,NaN,36539.0,12487.0,226.0,12261.0,1.809882,98.190118,23.830048,24.793549


In [175]:
df_full.to_csv('gas.csv')